In [8]:
import pandas as pd
from refine_func import *


In [9]:
# load data
data = pd.read_csv("../data/temp.csv")
data.reset_index(inplace=True)

In [10]:
# 명사 불용어 처리
stopwords = ["이", "거", "게", "개", "심", "듯", "좀", "함", "것"]
# 전처리
reviewList = data['review'].tolist()
tokens: list[list[str]] = extract_pos(reviewList, "Noun")
tokens = filter_stopword(tokens, stopwords)
frequency: dict =  count_frequency(tokens)
sorted_frequency = list(sorted(frequency, key=frequency.get, reverse=True))
for token in sorted_frequency[:100]:
    print(f"{token}: {frequency[token]}")


수업: 2471
교수: 1912
시험: 1761
강의: 1715
과제: 1317
문제: 1302
말: 1085
수: 1033
중간: 806
성적: 797
내용: 792
점: 737
학점: 713
공부: 690
생각: 645
안: 586
정도: 557
학기: 551
때: 528
시간: 526
점수: 502
족보: 496
정말: 456
번: 445
추천: 403
설명: 372
그냥: 365
저: 353
학생: 349
이해: 325
질문: 319
사람: 304
영어: 300
퀴즈: 267
하나: 265
진짜: 264
매우: 264
평균: 250
때문: 249
다른: 248
출석: 248
과목: 245
중: 245
분: 237
거의: 229
더: 228
주시: 225
부분: 224
이번: 223
개인: 220
만점: 216
교재: 216
그: 215
걸: 214
후: 213
조금: 211
전: 211
다만: 206
풀: 199
보고: 198
발표: 198
기말고사: 196
느낌: 196
난이도: 196
개념: 189
부담: 189
중간고사: 187
자체: 185
제: 184
본인: 183
로드: 178
대면: 174
처음: 170
필기: 166
책: 166
건: 162
꼭: 162
모두: 160
팀: 160
말씀: 154
임: 154
수강: 152
연습: 150
꿀강: 148
수준: 147
주심: 145
답: 145
꽤: 145
도움: 142
평가: 140
한번: 138
복습: 134
참여: 134
별로: 133
무난: 133
기준: 133
방식: 131
노력: 131
나: 130
못: 129


In [11]:
detokenized = deto(tokens)
print(detokenized[:10])

['강의 과제 제출 만점 시험 편이 학점 공지 칼', '과제 제출 만점 사실 시험 번 성적 결정 상대평가 시기 때문 득점 중간고사 때 점 기말고사 때 점 뜰 줄 기말고사 제공 연습 문제 제대로 준비 성적 다른 과목 편 교수 이해 강의', '강의 진행 성적 칼 비율 주시 개념 복습 용 과제 아무 부담', '교수 정말 강의 개인 생각 초반 통계학 입문 복습 느낌 통입 해 후반 분산 회귀분석 위주 교수 설명 수업 이해 수 합 로 에쁠 제 수 생각 에쁠 성적 빡빡 시진', '중간고사 통계학 입문 내용 분포 추정 표본 표본 검정 카이 제곱 검정 복습 기말고사 분산분석 선형회귀 분석 계절 내용 연결 성적 칼 불만 일 순 강의 성격 순 적응', '교수 수업 집중 복습 전혀 지장 과제 정도 과제 복습 성적 수 성적 비율 상대평가 주시 분반 별로 성적 산출 참고', '만큼 성적 공부', '통방 대해 공부 학점 나름 수', '중간 말 로 빡세 성적 컷 널널', '수업 저 통입 통입 선수 필수 대해 서도 기본 애 대해 수 교수 예제 분산분석 회귀분석 설명 시작 이해 문제 조금 느낌 수업 중 가끔 동기부여 차원 말씀 나름 전 도움 통방 정종 교수 수업 정말 과제 채점 정말 하나 하나 피드백 중간 말']


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(detokenized)
print(f"shape {X.shape}")


shape (2576, 1000)


In [27]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=6,learning_method='online',random_state=777,max_iter=1)
lda_top = lda_model.fit_transform(X)
print(lda_model.components_)
# print(lda_model.components_.shape) 


[[ 0.21524847  0.26910974  0.38240097 ...  0.23253751  0.29534642
   0.21508151]
 [ 0.2059427   0.216752    0.26228597 ...  0.21862711  0.29754329
   0.21803903]
 [ 2.93355173  6.83849688  8.25880809 ... 11.18164037  0.90122404
   0.82213618]
 [ 0.22127597  0.25317526  0.21190659 ...  0.3618148   0.21947957
   0.25290512]
 [ 0.21622115  0.49802288  0.21003649 ...  0.23559283  0.21632076
   0.2123223 ]
 [ 0.23370623  0.22149766  0.20515329 ...  0.21092057  0.21791601
   5.2106776 ]]


In [28]:
terms = vectorizer.get_feature_names_out()

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], float(topic[i].round(2))) for i in topic.argsort()[:-n - 1:-1]])
        # arr = topic.argsort()[:-n-1: -1]
        # print()

get_topics(lda_model.components_,terms)


Topic 1: [('영화', 13.37), ('졸업', 4.01), ('모형', 2.44), ('감상', 2.1), ('대한', 1.93), ('중구', 1.92), ('난방', 1.91), ('창작', 1.87), ('경험', 1.86), ('추상', 1.84)]
Topic 2: [('인공', 8.75), ('종류', 7.3), ('지능', 7.28), ('학점', 5.79), ('진짜', 5.76), ('교수', 3.87), ('코딩', 3.76), ('완전', 3.44), ('티어', 3.4), ('배경', 3.25)]
Topic 3: [('수업', 178.23), ('과제', 170.41), ('강의', 135.41), ('시험', 135.24), ('교수', 133.54), ('학점', 96.55), ('문제', 91.42), ('성적', 87.58), ('중간', 77.39), ('내용', 72.93)]
Topic 4: [('깊이', 8.2), ('널널', 7.01), ('사랑', 6.64), ('의성', 4.67), ('최악', 4.15), ('무슨', 3.52), ('교수', 3.03), ('성적', 2.97), ('제때', 2.81), ('등록금', 2.79)]
Topic 5: [('학점', 20.61), ('교수', 4.83), ('내용', 4.48), ('공대생', 4.35), ('연세대', 4.31), ('강사', 4.18), ('수업', 3.98), ('쁠몰', 3.87), ('추천', 3.76), ('성공', 3.7)]
Topic 6: [('교양', 28.97), ('개꿀강', 9.76), ('하라', 8.28), ('의심', 6.17), ('음악', 5.4), ('힐링', 5.21), ('성의', 3.72), ('공부', 3.1), ('교수', 2.68), ('감상문', 2.52)]
